In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor

### Instantiate function for each regression model 

In [7]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1e-4, 1000, log=True)
    }

    return Ridge(**params)

def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", 1e-4, 1000, log=True)
    }

    return Lasso(**params)

def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", 1e-4, 1000, log=True)
    }

    return ElasticNet(**params)

def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        "objective": "reg:squarederror",
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20)
    }

    return XGBRegressor(**params)

def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 1, 10),
    'min_samples_split': trial.suggest_int ('min_sample_split', 2, 10)
    }
    
    return GradientBoostingRegressor(**params)

# def instantiate_rf(trial : Trial) -> RandomForestRegressor:
#     params = {
#     'bootstrap':trial.suggest_categorical('bootstrap', [True, False]),
#     'max_features':trial.suggest_categorical('max_features', None, 'sqrt'),
#     'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#     'max_depth': trial.suggest_float('max_depth', 1, 10),
#     'min_samples_split': trial.suggest_float('min_sample_split', 2, 10),
#     'min_samples_leaf': trial.suggest_float('min_sample_leaf', 2, 10)
#     }
    
#     return RandomForestRegressor(**params)

# def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
#     params = {
#     'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
#     'loss': trial.suggest_categorical('loss',['linear', 'square', 'expoential']),
#     'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
#     }
    
#     return AdaBoostRegressor(**params)



### Instantiate function for regression model selection

In [8]:
Classifier = (
    Ridge |
    Lasso |
    XGBRegressor |
    GradientBoostingRegressor
    )

def instantiate_learner(trial : Trial) -> Classifier:
    algorithm = trial.suggest_categorical(
    'algorithm', ['ridge', 'lasso', 'xgb', 'gbr'])
    
    if algorithm =='ridge':
        model = instantiate_ridge(trial)
    elif algorithm=='lasso':
        model = instantiate_lasso(trial)
    elif algorithm=='xgb':
        model = instantiate_xgb(trial)
    elif algorithm=='gbr':
        model = instantiate_gbr(trial)
        
    return model

### Instantiate functions for encoding categorical columns 

In [161]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

### Instantiate functions for encoding numerical columns 

In [162]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

### Instantiate function to scale and encode 

In [163]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_learner(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

In [164]:
def objective(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [168]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [172]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize')

[I 2024-05-08 14:57:04,785] A new study created in memory with name: optimization


In [173]:
study.optimize(lambda trial: objective(trial, X, y), n_trials=200)

[I 2024-05-08 14:57:06,873] Trial 0 finished with value: -0.029441307066881618 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 485.91494069511094}. Best is trial 0 with value: -0.029441307066881618.
[I 2024-05-08 15:01:14,345] Trial 1 finished with value: 0.5189862940139655 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'gbr', 'n_estimators': 436, 'learning_rate': 0.0012861054183096832, 'max_depth': 8, 'min_sample_split': 7}. Best is trial 1 with value: 0.5189862940139655.
[I 2024-05-08 15:01:14,697] Trial 2 finished with value: 0.9408951773195543 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'ordinal', 'algorithm': 'ridge', 'alpha': 2.9564732483382494}. Best is trial 2 with value: 0.9408951773195543.
[I 2024-05-08 15:01:15,064] Trial 3 finished with value: 0.9224959498178851 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordin

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:06:41,074] Trial 18 finished with value: 0.6551588287903598 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': 'first', 'algorithm': 'gbr', 'n_estimators': 871, 'learning_rate': 0.0010131947131918157, 'max_depth': 5, 'min_sample_split': 2}. Best is trial 16 with value: 0.9499562491134823.
[I 2024-05-08 15:06:41,551] Trial 19 finished with value: 0.9494180201020687 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'ridge', 'alpha': 9.142237389730353}. Best is trial 16 with value: 0.9499562491134823.
[I 2024-05-08 15:06:42,100] Trial 20 finished with value: 0.9498433838413829 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'ridg

[I 2024-05-08 15:19:22,673] Trial 39 finished with value: 0.6705510918020474 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'ridge', 'alpha': 605.5367260873303}. Best is trial 32 with value: 0.9499609584764295.
[I 2024-05-08 15:19:32,409] Trial 40 finished with value: -0.663960973067949 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'xgb', 'learning_rate': 0.003149557674088468, 'max_depth': 4, 'subsample': 0.6126599257625832, 'colsample_bytree': 0.05233759359756812, 'min_child_weight': 12}. Best is trial 32 with value: 0.9499609584764295.
[I 2024-05-08 15:19:32,892] Trial 41 finished with value: 0.9500242977645403 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'ridge', 'alpha': 26.899525846925062}. Best is trial 41 with value: 0.9500242977645403.
[I 2024-05-08 15:19:33,443] Trial 42 finished with value: 0.9500184207578766 and 

[I 2024-05-08 15:23:30,146] Trial 61 finished with value: 0.9410978568883082 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.004005670923160421}. Best is trial 53 with value: 0.95003638254706.
[I 2024-05-08 15:23:31,084] Trial 62 finished with value: 0.9503991540555667 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00017988210177904444}. Best is trial 62 with value: 0.9503991540555667.
[I 2024-05-08 15:23:32,172] Trial 63 finished with value: 0.9500394892970639 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00011474761017558412}. Best is trial 62 with value: 0.9503991540555667.
[I 2024-05-08 15:23:33,254] Trial 64 finished with value: 0.9501568187856455 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha'

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:23:40,670] Trial 73 finished with value: 0.9504428094919153 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': 'first', 'algorithm': 'lasso', 'alpha': 0.00022755348542197014}. Best is trial 66 with value: 0.9507977505481847.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: Use

...

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:30:46,301] Trial 144 finished with value: 0.008035087384765282 and parameters: {'scaling_method': 'robust', 'encoding_method': '

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:30:51,585] Trial 151 finished with value: 0.9504395074763805 and parameters: {'scaling_method': 'robust', 'encoding_method': 'on

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:30:56,716] Trial 157 finished with value: 0.9454677865190133 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': 'first', 'algorithm': 'lasso', 'alpha': 0.0014317539569420536}. Best is trial 66 with value: 0.9507977505481847.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWa

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-08 15:33:57,223] Trial 164 finished with value: 0.9505364077318952 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': 'first', 'algorithm': 'lasso', 'alpha': 0.0003519631738583915}. Best is trial 66 with value: 0.9507977505481847.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWa

[I 2024-05-08 15:34:15,488] Trial 176 finished with value: 0.9502475455944255 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00015123432538843447}. Best is trial 169 with value: 0.9508124784027794.
[I 2024-05-08 15:34:16,200] Trial 177 finished with value: 0.9502968072315191 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0002785330774893725}. Best is trial 169 with value: 0.9508124784027794.
[I 2024-05-08 15:34:16,754] Trial 178 finished with value: 0.94179955413324 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'algorithm': 'lasso', 'alpha': 0.0004111881257917249}. Best is trial 169 with value: 0.9508124784027794.
[I 2024-05-08 15:34:17,748] Trial 179 finished with value: 0.9504084264437751 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.

In [174]:
study.best_trial

FrozenTrial(number=169, state=1, values=[0.9508124784027794], datetime_start=datetime.datetime(2024, 5, 8, 15, 34, 9, 247763), datetime_complete=datetime.datetime(2024, 5, 8, 15, 34, 10, 53415), params={'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003052236269302166}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'scaling_method': CategoricalDistribution(choices=('standard', 'minmax', 'maxabs', 'robust')), 'encoding_method': CategoricalDistribution(choices=('ordinal', 'onehot')), 'drop': CategoricalDistribution(choices=(None, 'first')), 'algorithm': CategoricalDistribution(choices=('ridge', 'lasso', 'xgb', 'gbr')), 'alpha': FloatDistribution(high=1000.0, log=True, low=0.0001, step=None)}, trial_id=169, value=None)

In [180]:
trial_number = []
score = []
algorithm = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study.get_trials()[trial].number)
    score.append(study.get_trials()[trial].value)
    algorithm.append(study.get_trials()[trial].params['algorithm'])
    parameters.append(list(study.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Algorithm'] = algorithm
optuna_dict['Parameters'] = parameters


df_optuna = pd.DataFrame.from_dict(optuna_dict)

In [181]:
idx = df_optuna.groupby('Algorithm')['Score'].transform(max) == df_optuna['Score']
df_optuna[idx]

,Trial,Score,Algorithm,Parameters
53,53,0.950036,ridge,"[(scaling_method, robust), (encoding_method, o..."
135,135,0.941817,gbr,"[(scaling_method, robust), (encoding_method, o..."
137,137,0.947552,xgb,"[(scaling_method, robust), (encoding_method, o..."
169,169,0.950812,lasso,"[(scaling_method, robust), (encoding_method, o..."


In [191]:
list(df_optuna.Parameters.items())

[(0,
  [('scaling_method', 'robust'),
   ('encoding_method', 'onehot'),
   ('drop', None),
   ('algorithm', 'lasso'),
   ('alpha', 485.91494069511094)]),
 (1,
  [('scaling_method', 'minmax'),
   ('encoding_method', 'onehot'),
   ('drop', None),
   ('algorithm', 'gbr'),
   ('n_estimators', 436),
   ('learning_rate', 0.0012861054183096832),
   ('max_depth', 8),
   ('min_sample_split', 7)]),
 (2,
  [('scaling_method', 'minmax'),
   ('encoding_method', 'ordinal'),
   ('algorithm', 'ridge'),
   ('alpha', 2.9564732483382494)]),
 (3,
  [('scaling_method', 'maxabs'),
   ('encoding_method', 'ordinal'),
   ('algorithm', 'ridge'),
   ('alpha', 17.622180822136105)]),
 (4,
  [('scaling_method', 'standard'),
   ('encoding_method', 'onehot'),
   ('drop', None),
   ('algorithm', 'ridge'),
   ('alpha', 11.016938443949593)]),
 (5,
  [('scaling_method', 'minmax'),
   ('encoding_method', 'onehot'),
   ('drop', 'first'),
   ('algorithm', 'ridge'),
   ('alpha', 72.72185663024044)]),
 (6,
  [('scaling_method

In [193]:
df_optuna[df_optuna.Algorithm == 'lasso']

,Trial,Score,Algorithm,Parameters
0,0,-0.029441,lasso,"[(scaling_method, robust), (encoding_method, o..."
7,7,0.158099,lasso,"[(scaling_method, maxabs), (encoding_method, o..."
9,9,-0.029441,lasso,"[(scaling_method, maxabs), (encoding_method, o..."
28,28,-0.029441,lasso,"[(scaling_method, robust), (encoding_method, o..."
29,29,-0.029441,lasso,"[(scaling_method, robust), (encoding_method, o..."
...,...,...,...,...
193,193,0.950580,lasso,"[(scaling_method, robust), (encoding_method, o..."
194,194,0.943710,lasso,"[(scaling_method, robust), (encoding_method, o..."
195,195,0.948354,lasso,"[(scaling_method, robust), (encoding_method, o..."
196,196,0.950578,lasso,"[(scaling_method, robust), (encoding_method, o..."


In [182]:
print('lasso', df_optuna.Parameters[169])
print('ridge', df_optuna.Parameters[53])
print('xgb', df_optuna.Parameters[137])
print('gbr', df_optuna.Parameters[135])

lasso [('scaling_method', 'robust'), ('encoding_method', 'onehot'), ('drop', None), ('algorithm', 'lasso'), ('alpha', 0.0003052236269302166)]
ridge [('scaling_method', 'robust'), ('encoding_method', 'onehot'), ('drop', None), ('algorithm', 'ridge'), ('alpha', 19.535539796579627)]
xgb [('scaling_method', 'robust'), ('encoding_method', 'onehot'), ('drop', 'first'), ('algorithm', 'xgb'), ('learning_rate', 0.04284827303418196), ('max_depth', 6), ('subsample', 0.7820149753016207), ('colsample_bytree', 0.7705630302036239), ('min_child_weight', 16)]
gbr [('scaling_method', 'robust'), ('encoding_method', 'onehot'), ('drop', 'first'), ('algorithm', 'gbr'), ('n_estimators', 723), ('learning_rate', 0.023568214077863484), ('max_depth', 4), ('min_sample_split', 8)]


In [5]:
import plotly.express as px


fig = px.scatter(df_optuna.loc[df_optuna.Score > .9], 
                 x="Trial", 
                 y="Score", 
                 color="Algorithm",
                 hover_data=['Parameters'])

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

NameError: name 'df_optuna' is not defined